In [3]:
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
from src.dataloader import create_dataloader
from src.subset_dataloader import sub_create_dataloader
from src.model import Model
from src.utils.torch_utils import model_info, check_runtime
from src.trainer import TorchTrainer, count_model_params
from typing import Any, Dict, List, Tuple
from optuna.pruners import HyperbandPruner
from subprocess import _args_from_interpreter_flags
import argparse

In [4]:
if not torch.cuda.is_available():
    device = torch.device("cpu")
else:
    device = torch.device(f"cuda:0")

In [5]:
import argparse
import os
from datetime import datetime
from typing import Any, Dict, Tuple, Union

import torch
import torch.nn as nn
import torch.optim as optim
import yaml

from src.dataloader import create_dataloader
from src.loss import CustomCriterion
from src.model import Model
from src.trainer import TorchTrainer
from src.utils.common import get_label_counts, read_yaml
from src.utils.torch_utils import check_runtime, model_info

In [6]:
DATA_PATH = '../data'
RESULT_MODEL_PATH = './sub_small.pt'

In [7]:
epochs = 50
img_size = 96
n_select = None
batch_size = 32 

In [8]:
def search_model(trial: optuna.trial.Trial) -> List[Any]:
    """Search model structure from user-specified search space."""
    model = []
    n_stride = 0
    MAX_NUM_STRIDE = 5
    UPPER_STRIDE = 2  # 5(224 example): 224, 112, 56, 28, 14, 7
    n_layers = trial.suggest_int("n_layers", 8, 12)
    stride = 1
    input_max = 64
    imput_min = 32
    module_info = {}
    ### 몇개의 레이어를 쌓을지도 search하게 했습니다.
    for i in range(n_layers):
        out_channel = trial.suggest_int(f"{i+1}units", imput_min, input_max)
        block = trial.suggest_categorical(
            f"m{i+1}", ["Conv", "DWConv", "InvertedResidualv2", "InvertedResidualv3"]
        )
        repeat = trial.suggest_int(f"m{i+1}/repeat", 1, 5)
        m_stride = trial.suggest_int(f"m{i+1}/stride", low=1, high=UPPER_STRIDE)
        if m_stride == 2:
            stride += 1
        if n_stride == 0:
            m_stride = 2

        if block == "Conv":
            activation = trial.suggest_categorical(f"m{i+1}/activation", ["ReLU", "Hardswish"])
            # Conv args: [out_channel, kernel_size, stride, padding, groups, activation]
            args = [out_channel, 3, m_stride, None, 1, activation]
        elif block == "DWConv":
            activation = trial.suggest_categorical(f"m{i+1}/activation", ["ReLU", "Hardswish"])
            # DWConv args: [out_channel, kernel_size, stride, padding_size, activation]
            args = [out_channel, 3, 1, None, activation]
        elif block == "InvertedResidualv2":
            c = trial.suggest_int(f"m{i+1}/v2_c", low=imput_min, high=input_max, step=16)
            t = trial.suggest_int(f"m{i+1}/v2_t", low=1, high=4)
            args = [c, t, m_stride]
        elif block == "InvertedResidualv3":
            kernel = trial.suggest_int(f"m{i+1}/kernel_size", low=3, high=5, step=2)
            t = round(trial.suggest_float(f"m{i+1}/v3_t", low=1.0, high=6.0, step=0.1), 1)
            c = trial.suggest_int(f"m{i+1}/v3_c", low=imput_min, high=input_max, step=8)
            se = trial.suggest_categorical(f"m{i+1}/v3_se", [0, 1])
            hs = trial.suggest_categorical(f"m{i+1}/v3_hs", [0, 1])
            # k t c SE HS s
            args = [kernel, t, c, se, hs, m_stride]

        in_features = out_channel
        model.append([repeat, block, args])
        if i % 2:
            input_max *= 2
            input_max = min(input_max, 160)
        module_info[f"block{i+1}"] = {"type": block, "repeat": repeat, "stride": stride}
    # last layer
    last_dim = trial.suggest_int("last_dim", low=128, high=1024, step=128)
    # We can setup fixed structure as well
    model.append([1, "Conv", [last_dim, 1, 1]])
    model.append([1, "GlobalAvgPool", []])
    model.append([1, "FixedConv", [6, 1, 1, None, 1, None]])
    return model, module_info

In [9]:
def objective(trial: optuna.trial.Trial, device, fp16) -> Tuple[float, int, float]:
    """Optuna objective.
    Args:
        trial
    Returns:
        float: score1(e.g. accuracy)
        int: score2(e.g. params)
    """
    model_config: Dict[str, Any] = {}
    model_config["input_channel"] = 3
    # img_size = trial.suggest_categorical("img_size", [32, 64, 128])
    img_size = 32
    model_config["INPUT_SIZE"] = [img_size, img_size]
    model_config["depth_multiple"] = trial.suggest_categorical(
        "depth_multiple", [0.25, 0.5, 0.75, 1.0]
    )
    model_config["width_multiple"] = trial.suggest_categorical(
        "width_multiple", [0.25, 0.5, 0.75, 1.0]
    )
    model_config["backbone"], module_info = search_model(trial)
    hyperparams = {
        "EPOCHS" : epochs,
        "IMG_SIZE" : img_size,
        "n_select" : n_select, # ??
        "BATCH_SIZE" : batch_size 
    }

    model = Model(model_config, verbose=True)
    model.to(device)
    model.model.to(device)

    # check ./data_configs/data.yaml for config information
    data_config: Dict[str, Any] = {}
    data_config["DATA_PATH"] = DATA_PATH
    data_config["DATASET"] = "TACO"
    data_config["AUG_TRAIN"] = "simple_augment_train" # default: randaugment_train
    data_config["AUG_TEST"] = "simple_augment_test"
    # data_config["AUG_TRAIN_PARAMS"] = {
    #     "n_select": hyperparams["n_select"],
    # }
    data_config['AUG_TRAIN_PARAMS'] = None
    data_config["AUG_TEST_PARAMS"] = None
    data_config["BATCH_SIZE"] = hyperparams["BATCH_SIZE"]
    data_config["VAL_RATIO"] = 0.8
    data_config["IMG_SIZE"] = hyperparams["IMG_SIZE"]
    data_config["INIT_LR"] = 0.1
    data_config["EPOCHS"] = 100
    """이부분이 config를 저장하는 부분입니다. 위의 lr,fp16,epochs는 원래 함수에는 없지만
    config를 바로 사용할 수 있게 추가했습니다."""
    
    k = 1
    file_name = f"search_model/model_{k}.yaml"
    while os.path.exists(file_name):
        print(k)
        k += 1
        file_name = f"search_model/model_{k}.yaml"
    print(model_config)
    "model config와 data config를 저장"
    with open(f"search_model/model_{k}.yaml", "w") as outfile:
        yaml.dump(model_config, outfile)
    with open(f"search_model/data_{k}.yaml", "w") as outfile:
        yaml.dump(data_config, outfile)

    mean_time = check_runtime(
        model.model,
        [model_config["input_channel"]] + model_config["INPUT_SIZE"],
        device,
    )
    model_info(model, verbose=True)
    train_loader, val_loader, test_loader = sub_create_dataloader(data_config)

    # criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=0.1,
        steps_per_epoch=len(train_loader),
        epochs=hyperparams["EPOCHS"],
        pct_start=0.05,
    )

    criterion = CustomCriterion(
        samples_per_cls=get_label_counts(data_config["DATA_PATH"])
        if data_config["DATASET"] == "TACO"
        else None,
        device=device,
    )
    # Amp loss scaler
    scaler = (
        torch.cuda.amp.GradScaler() if fp16 and device != torch.device("cpu") else None
    )
    # scaler = torch.cuda.amp.GradScaler() if fp16 and device != torch.device("cpu") else None

    trainer = TorchTrainer(
        model,
        criterion,
        optimizer,
        scheduler,
        device=device,
        verbose=1,
        model_path=RESULT_MODEL_PATH,
        scaler=scaler,
    )
    trainer.train(train_loader, hyperparams["EPOCHS"], val_dataloader=val_loader)
    loss, f1_score, acc_percent = trainer.test(model, test_dataloader=val_loader)
    params_nums = count_model_params(model)

    model_info(model, verbose=True)
    return f1_score, params_nums, mean_time

In [10]:
def get_best_trial_with_condition(optuna_study: optuna.study.Study) -> Dict[str, Any]:
    """Get best trial that satisfies the minimum condition(e.g. accuracy > 0.8).
    Args:
        study : Optuna study object to get trial.
    Returns:
        best_trial : Best trial that satisfies condition.
    """
    df = optuna_study.trials_dataframe().rename(
        columns={
            "values_0": "acc_percent",
            "values_1": "params_nums",
            "values_2": "mean_time",
        }
    )
    ## minimum condition : accuracy >= threshold
    threshold = 0.7
    minimum_cond = df.acc_percent >= threshold

    if minimum_cond.any():
        df_min_cond = df.loc[minimum_cond]
        ## get the best trial idx with lowest parameter numbers
        best_idx = df_min_cond.loc[
            df_min_cond.params_nums == df_min_cond.params_nums.min()
        ].acc_percent.idxmax()

        best_trial_ = optuna_study.trials[best_idx]
        print("Best trial which satisfies the condition")
        print(df.loc[best_idx])
    else:
        print("No trials satisfies minimum condition")
        best_trial_ = None

    return best_trial_

In [11]:
def tune(gpu_id, storage: str = None, fp16: bool = False):
    if not torch.cuda.is_available():
        device = torch.device("cpu")
    elif 0 <= gpu_id < torch.cuda.device_count():
        device = torch.device(f"cuda:{gpu_id}")
    sampler = optuna.samplers.MOTPESampler()
    if storage is not None:
        rdb_storage = optuna.storages.RDBStorage(url=storage)
    else:
        rdb_storage = None
    study = optuna.create_study(
        directions=["maximize", "minimize", "minimize"],
        study_name="automl",
        sampler=sampler,
        storage=rdb_storage,
        load_if_exists=True,
    )
    study.optimize(lambda trial: objective(trial, device, fp16), n_trials=30)
    pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
    complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trials:")
    best_trials = study.best_trials

    ## trials that satisfies Pareto Fronts
    for tr in best_trials:
        print(f"  value1:{tr.values[0]}, value2:{tr.values[1]}")
        for key, value in tr.params.items():
            print(f"    {key}:{value}")

    best_trial = get_best_trial_with_condition(study)
    print(best_trial)
    df = study.trials_dataframe(attrs=("number", "value", "params", "state"))
    df.to_csv("search_results.csv", index=False)

In [12]:
parser = argparse.ArgumentParser(description="Optuna tuner.")
parser.add_argument("--gpu", default=0, type=int, help="GPU id to use")
parser.add_argument("--storage", default="", type=str, help="Optuna database storage path.")
parser.add_argument("--fp16", default=False, type=bool, help="train to fp16")
args = parser.parse_args([])  
tune(args.gpu, storage=args.storage if args.storage != "" else None, fp16=args.fp16)

<ipython-input-11-e99111987fa0>:6: ExperimentalWarning: MOTPESampler is experimental (supported from v2.4.0). The interface can change in the future.
  sampler = optuna.samplers.MOTPESampler()
[I 2021-11-27 07:36:56,520] A new study created in memory with name: automl


idx |   n |     params |          module |            arguments |   in_channel |   out_channel
----------------------------------------------------------------------------------------------
  0 |   1 |        352 | InvertedResidualv3 | [3, 1.4, 32, 1, 0, 2] |            3            8
  1 |   1 |      1,728 | InvertedResidualv3 | [5, 3.4, 64, 0, 1, 2] |            8           16
  2 |   1 |      2,384 | InvertedResidualv2 |           [32, 4, 2] |           16            8
  3 |   1 |      1,184 |            Conv | [55, 3, 2, None, 1, 'Hardswish'] |            8           16
  4 |   1 |      2,336 |            Conv | [40, 3, 2, None, 1, 'Hardswish'] |           16           16
  5 |   1 |      5,760 | InvertedResidualv3 | [5, 3.0, 144, 1, 0, 2] |           16           40
  6 |   1 |      1,128 |          DWConv | [106, 3, 1, None, 'ReLU'] |           40           24
  7 |   1 |      6,136 | InvertedResidualv3 | [3, 3.8, 128, 0, 0, 2] |           24           32
  8 |   1 |      9,280 |

Train: [001] Loss: 1.650, Acc: 31.89% F1(macro): 0.16: 100%|██████████| 146/146 [00:22<00:00,  6.47it/s]
 Val:       Loss: 1.527, Acc: 37.00% F1(macro): 0.16: 100%|██████████| 163/163 [00:28<00:00,  5.72it/s]


Model saved. Current best test f1: 0.163


Train: [002] Loss: 1.533, Acc: 37.35% F1(macro): 0.19: 100%|██████████| 146/146 [00:26<00:00,  5.46it/s]
 Val:       Loss: 1.563, Acc: 36.17% F1(macro): 0.15: 100%|██████████| 163/163 [00:29<00:00,  5.48it/s]
Train: [003] Loss: 1.536, Acc: 36.99% F1(macro): 0.17: 100%|██████████| 146/146 [00:21<00:00,  6.85it/s]
 Val:       Loss: 1.518, Acc: 38.17% F1(macro): 0.16: 100%|██████████| 163/163 [00:33<00:00,  4.87it/s]
Train: [004] Loss: 1.526, Acc: 37.18% F1(macro): 0.18: 100%|██████████| 146/146 [00:19<00:00,  7.57it/s]
 Val:       Loss: 1.527, Acc: 36.77% F1(macro): 0.18: 100%|██████████| 163/163 [00:36<00:00,  4.47it/s]


...

Model saved. Current best test f1: 0.275


Train: [043] Loss: 1.359, Acc: 47.00% F1(macro): 0.29: 100%|██████████| 146/146 [00:14<00:00, 10.38it/s]
 Val:       Loss: 1.373, Acc: 46.82% F1(macro): 0.27: 100%|██████████| 163/163 [00:23<00:00,  6.81it/s]
Train: [044] Loss: 1.346, Acc: 47.24% F1(macro): 0.31: 100%|██████████| 146/146 [00:14<00:00, 10.02it/s]
 Val:       Loss: 1.377, Acc: 46.73% F1(macro): 0.26: 100%|██████████| 163/163 [00:24<00:00,  6.77it/s]
Train: [045] Loss: 1.346, Acc: 48.05% F1(macro): 0.29: 100%|██████████| 146/146 [00:13<00:00, 10.61it/s]
 Val:       Loss: 1.374, Acc: 47.02% F1(macro): 0.27: 100%|██████████| 163/163 [00:23<00:00,  6.81it/s]
Train: [046] Loss: 1.344, Acc: 48.20% F1(macro): 0.30: 100%|██████████| 146/146 [00:14<00:00, 10.40it/s]
 Val:       Loss: 1.368, Acc: 47.36% F1(macro): 0.29: 100%|██████████| 163/163 [00:24<00:00,  6.78it/s]


Model saved. Current best test f1: 0.286


Train: [047] Loss: 1.347, Acc: 48.39% F1(macro): 0.31: 100%|██████████| 146/146 [00:14<00:00, 10.37it/s]
 Val:       Loss: 1.368, Acc: 47.40% F1(macro): 0.28: 100%|██████████| 163/163 [00:23<00:00,  6.80it/s]
Train: [048] Loss: 1.317, Acc: 48.76% F1(macro): 0.31: 100%|██████████| 146/146 [00:14<00:00,  9.97it/s]
 Val:       Loss: 1.368, Acc: 47.28% F1(macro): 0.27: 100%|██████████| 163/163 [00:23<00:00,  6.83it/s]
Train: [049] Loss: 1.327, Acc: 49.29% F1(macro): 0.32: 100%|██████████| 146/146 [00:13<00:00, 10.45it/s]
 Val:       Loss: 1.363, Acc: 47.40% F1(macro): 0.27: 100%|██████████| 163/163 [00:24<00:00,  6.79it/s]
Train: [050] Loss: 1.323, Acc: 49.10% F1(macro): 0.31: 100%|██████████| 146/146 [00:14<00:00, 10.38it/s]
 Val:       Loss: 1.368, Acc: 46.88% F1(macro): 0.28: 100%|██████████| 163/163 [00:23<00:00,  6.82it/s]
 Val:       Loss: 1.368, Acc: 46.88% F1(macro): 0.28: 100%|██████████| 163/163 [00:23<00:00,  6.81it/s]
[I 2021-11-28 01:04:26,854] Trial 29 finished with values: [

layer                                     name  gradient   parameters                shape         mu      sigma
    0                  model.0.0.conv.0.weight      True           48        [16, 3, 1, 1]      -0.14       1.91
    1                  model.0.0.conv.1.weight      True           16                 [16]       1.07      0.256
    2                    model.0.0.conv.1.bias      True           16                 [16]    -0.0311      0.597
    3                  model.0.0.conv.3.weight      True          400        [16, 1, 5, 5]      0.214      0.755
    4                  model.0.0.conv.4.weight      True           16                 [16]       1.03      0.369
    5                    model.0.0.conv.4.bias      True           16                 [16]      0.186      0.414
    6              model.0.0.conv.5.fc1.weight      True          128        [8, 16, 1, 1]     0.0191      0.249
    7                model.0.0.conv.5.fc1.bias      True            8                  [8]     0

In [1]:
1.0399571774683296e-05 * 10

0.00010399571774683297